<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ken_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KenLM Framework for Language Modeling


**Install KenLM**

Download stable release and unzip: http://kheafield.com/code/kenlm.tar.gz

Need Boost >= 1.42.0 and bjam
*   Ubuntu: sudo apt-get install libboost-all-dev
*   Mac: brew install boost; brew install bjam

Run within kenlm directory:
    
*  mkdir -p build
  *  cd build
  *  cmake ..
  *  make -j 4
 
pip install https://github.com/kpu/kenlm/archive/master.zip

For more information on KenLM see: https://github.com/kpu/kenlm and http://kheafield.com/code/kenlm/

TODO: need to get some trained language models: nli_5gram.binary and sentiment.binary. How should we do that? 



In [1]:
import kenlm
import os

ModuleNotFoundError: ignored

Load the models

In [2]:

nli_model = kenlm.Model("nli_5gram.binary") # Or correct path to binary
sent_model =  kenlm.Model("sentiment.binary")

NameError: ignored

The KenLM model reports negative log likelihood, not perplexity. So we'll be converting the score and report net perplexity. The following function calculate the perpelxity, get_ppl, and find all OOV words, get_oov.

Pereplexity is defined as follows, $$ PPL = b^{- \frac{1}{N} \sum_{i=1}^N \log_b q(x_i)} $$ All probabilities here are in log base 10 so to convert to perplexity, we do the following $$PPL = 10^{-\log(P) / N} $$ where $P$ is the total NLL, and $N$ is the word count.

In [ ]:
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        sent = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", sent)
        words = sent.strip().split()
        score = lm.score(sent, bos=False, eos=False)
        word_count = len(words)
        total_wc += word_count
        total_nll += score
    ppl = 10**-(total_nll/total_wc)
    return ppl

def get_oov(model, data):
    oov = []
    vocab = []
    for sent in data:
        sentence = sent
        words =  sentence.split()
        vocab += words
        # Find out-of-vocabulary words
        for w in words:
            if w not in model:
                    oov.append(w)
    return set(oov), set(vocab)

Function for loading the data

In [ ]:
def load_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            data.append(line)
    return data

To start to get a feel for how the model performs on different sentences, try out different sentences below. (Note that this score isn't the perplexity but the negative log likelihood, you can convert it to perplexity as we did before)

In [ ]:
sentence = "I am a chimpanzee ."
print(nli_model.score(sentence))
print(sent_model.score(sentence))
